In [1]:
!pip install roboflow
                

/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'files.pythonhosted.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
     |████████████████████████████████| 89 kB 1.4 MB/s eta 0:00:01
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warn

In [3]:
pip install ultralytics

/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'files.pythonhosted.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
     |████████████████████████████████| 1.1 MB 1.4 MB/s eta 0:00:01
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-war

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="TcD1vumBq771nyc212qT")
project = rf.workspace("gauthams-workspace").project("my-first-project-4sbu2")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [3]:
dataset.location

'/home/gauthambharati/SmartSense/My-First-Project-2'

In [4]:
data_yaml_path = "/home/gauthambharati/SmartSense/My-First-Project-2/data.yaml"

In [5]:
import os
data_yaml_path = os.path.join(dataset.location, "data.yaml")

In [6]:
from ultralytics import YOLO

In [7]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'

In [8]:
def train_floorplan_model():

    print(f"Starting training... Using data from: {data_yaml_path}")

    model = YOLO('yolov8m.pt') 

    results = model.train(
        data=data_yaml_path,
        epochs=75,
        imgsz=512,
        project="floorplan_training",
        name="run_1",
    )

    print("Training complete.")
    save_dir = model.trainer.save_dir
    print(f"Model saved to: {save_dir}")
    print("Your best model is in: {save_dir}/weights/best.pt")


if not os.path.exists(data_yaml_path):
    print(f"Error: Could not find '{data_yaml_path}'.")
    print("Please download your dataset from Roboflow and update the 'data_yaml_path' variable.")
else:
    train_floorplan_model()

Starting training... Using data from: /home/gauthambharati/SmartSense/My-First-Project-2/data.yaml
Ultralytics 8.3.226 🚀 Python-3.8.10 torch-2.4.1+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32494MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/gauthambharati/SmartSense/My-First-Project-2/data.yaml, degrees=0.0, deterministic=True, device=2,3, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=75, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, na

In [12]:
import json
from collections import defaultdict

MODEL_PATH = "/home/gauthambharati/SmartSense/floorplan_training/run_1/weights/best.pt"

CLASS_MAPPING = {
    'Room': 'rooms',
    'bedroom': 'rooms',
    'bathroom': 'bathrooms',
    'hall': 'halls',
    'kitchen': 'kitchens',
    
    'garage': 'other',
    'laundary': 'other',
    'objects': 'other',
    'porch': 'other'
}

def parse_floorplan(image_path: str) -> str:
    try:
        model = YOLO(MODEL_PATH)
    except FileNotFoundError:
        print(f"Error: Model file not found at {MODEL_PATH}")
        print("Please make sure you have run train.py and the path is correct.")
        return json.dumps({
            "error": f"Model file not found at {MODEL_PATH}",
            "rooms": 0, "halls": 0, "kitchens": 0, "bathrooms": 0,
            "rooms_detail": []
        })
    except Exception as e:
         return json.dumps({"error": f"Error loading model: {e}"})

    print(f"Parsing {image_path}...")
    
    try:
        results = model(image_path)
    except Exception as e:
        print(f"Error during model inference: {e}")
        return json.dumps({"error": str(e)})

    model_class_names = model.names
    
    final_counts = defaultdict(int)
    
    room_details_counter = defaultdict(lambda: {"label": "", "count": 0, "approx_area": None})

    for result in results:
        if result.boxes:
            for box in result.boxes:
                class_index = int(box.cls)
                detected_class_name = model_class_names[class_index]
                
                mapped_class_name = CLASS_MAPPING.get(detected_class_name, 'other')
                
                if mapped_class_name != 'other':
                    final_counts[mapped_class_name] += 1
                

                detail_key = detected_class_name
                room_details_counter[detail_key]["label"] = detected_class_name
                room_details_counter[detail_key]["count"] += 1


    output = {
        "rooms": final_counts.get('rooms', 0),
        "halls": final_counts.get('halls', 0),
        "kitchens": final_counts.get('kitchens', 0),
        "bathrooms": final_counts.get('bathrooms', 0),
        "rooms_detail": list(room_details_counter.values()) 
    }
    
    return json.dumps(output, indent=2)


if not os.path.exists(MODEL_PATH):
    print(f"--- WARNING ---")
    print(f"Model file {MODEL_PATH} not found.")
    print("Please train your model and update MODEL_PATH variable at the top of this script.")
    print("Running with a placeholder image will fail.")
    print("-----------------")

example_image_path = "/home/gauthambharati/SmartSense/My-First-Project-2/test/images/72_12_jpg.rf.bba9499deff6484c0b57ee96271cd5a4.jpg" 

if os.path.exists(example_image_path):
    json_output = parse_floorplan(example_image_path)
    print("\n--- Output JSON ---")
    print(json_output)
else:
    print(f"\nError: Example image '{example_image_path}' not found.")
    print("Please update 'example_image_path' to test the script.")

Parsing /home/gauthambharati/SmartSense/My-First-Project-2/test/images/72_12_jpg.rf.bba9499deff6484c0b57ee96271cd5a4.jpg...

image 1/1 /home/gauthambharati/SmartSense/My-First-Project-2/test/images/72_12_jpg.rf.bba9499deff6484c0b57ee96271cd5a4.jpg: 512x512 3 Rooms, 1 bedroom, 1 hall, 1 kitchen, 8.0ms
Speed: 2.5ms preprocess, 8.0ms inference, 3.2ms postprocess per image at shape (1, 3, 512, 512)

--- Output JSON ---
{
  "rooms": 4,
  "halls": 1,
  "kitchens": 1,
  "bathrooms": 0,
  "rooms_detail": [
    {
      "label": "hall",
      "count": 1,
      "approx_area": null
    },
    {
      "label": "Room",
      "count": 3,
      "approx_area": null
    },
    {
      "label": "bedroom",
      "count": 1,
      "approx_area": null
    },
    {
      "label": "kitchen",
      "count": 1,
      "approx_area": null
    }
  ]
}
